In [63]:
import pandas as pd
import openai
import os
import json
import tiktoken
#from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from tqdm import tqdm
# Open AI API Key
openai.api_key = "YOUR_SECRET_KEY"

In [64]:
#function for API request
# we want the model to return a JSON object
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0.7,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        response_format={"type": "json_object"}
    )
    return json.loads(response.choices[0].message["content"])

In [65]:
#function to count the number of tokens
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
def count_tokens(text) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = len(encoding.encode(text))
    return num_tokens

In [74]:
# create a rationale given a news article
def get_rationale(article):
    impersonator = "You are an expert AI fact checker trained to detect fake news."
    instructor = "Is this article real or fake news? Analyze the given text in detail as a fact checker would. Explain your reasoning step-by-step. Check for misleading information, false claims, biased language. If the news article is real, respond with 1 for 'True', if fake, respond with 0 for 'False'."
    cloze_prompt = "Structure your answer as a JSON object with the two keys 'rationale' for your analytic reasoning and 'prediction' for your final prediction. The values for 'prediction' are binary. Either 1 for a real article or 0 for fake news. In 'rationale' you provide a string with your detailed reasoning process that led you to the decision you took. Remember if the article contains misleading information, false claims, biased language or any other hints of disinformation your prediction should be 0 for fake news, else 1 for true news. To summarize the JSON object looks like this: {'rationale': <your_analytic_reasoning_process>, 'prediction': <your_prediction[0,1]>} "
    prompt = f"{instructor} {cloze_prompt}"
    system_message = impersonator
    user_message = prompt +"Here is the article for you to analyze: "+ article

    messages =  [
        {'role':'system',
         'content': system_message},
        {'role':'user',
         'content': user_message},
    ]
    input = user_message + system_message
    input_tokens = count_tokens(input)
    response = get_completion_from_messages(messages)
    #print(response)
    model_prediction = response.get("prediction")
    rationale = response.get('rationale')
    return model_prediction, rationale, input_tokens

In [69]:
#example
get_rationale()

{'rationale': 'The given text does not present any clear indicators of fake news. It discusses a real initiative by Rob Corddry, a known comedian, to pay tribute to medical workers treating COVID-19. The information provided seems plausible and there are no obvious false claims or misleading information in the text.', 'prediction': 1}


(1,
 'The given text does not present any clear indicators of fake news. It discusses a real initiative by Rob Corddry, a known comedian, to pay tribute to medical workers treating COVID-19. The information provided seems plausible and there are no obvious false claims or misleading information in the text.',
 344)

In [72]:
def extend_data(dataframe, filename, max_retries=5):
    """
    :param dataframe: the dataset you want the model to create rationales for
    IMPORTANT: thsi dataframe needs to have a column called "text" which contains the news articles
    :param filename: the path and filename to save you csv file
    :param max_retries: how often you want to retry the API request in case something went wrong
    """
    model_predictions = []
    rationales = []
    input_tokens = []
    for i in tqdm(range(len(dataframe))):
        for attempt in range(max_retries):
            try:
                prediction, rationale, tokens = get_rationale(dataframe.text[i])
                model_predictions.append(prediction)
                rationales.append(rationale)
                input_tokens.append(tokens)
                break
            except Exception as e:
                print(f"Error: {e}. Attempt {attempt+1} of {max_retries}. Retrying...")
        else:
            print(f"Max retries exceeded for index {i}. Request failed.")
            model_predictions.append("Error")
            rationales.append("Error")
            input_tokens.append("Error")


    extended_dataframe = dataframe.assign(model_prediction = model_predictions, rationale = rationales, tokens_for_request = input_tokens)
    # Save the DataFrame as a CSV file
    extended_dataframe.to_csv(filename, index=True)